In [ ]:
# -------------------------------------------------------
# Global Configuration / Parameters
# -------------------------------------------------------

# The SQL Analytics Endpoint of your Fabric Lakehouse or Warehouse
SQL_ENDPOINT = "ABCXYZ123.datawarehouse.fabric.microsoft.com"

# The Database Name (Lakehouse or Warehouse name)
DATABASE_NAME = "bc2fabric_mirror"

# The Company Name filter for the Power BI Report
COMPANY_NAME = "CRONUS DE"

# Finance SQL Views Installer

This notebook fetches the latest SQL definition from GitHub and executes it against the specified SQL Analytics Endpoint.

**GitHub Source:** [finance-sql-views.sql](https://github.com/navida-Informationssysteme/BC2Fabric-Toolbox/blob/main/Accelerators/Finance-Reporting/finance-sql-views.sql)

In [ ]:
import requests
import pyodbc
import struct
import re

# Try importing the modern notebookutils, fallback to mssparkutils if needed
try:
    from notebookutils import credentials
except ImportError:
    from mssparkutils import credentials

# -------------------------------------------------------
# 1. Fetch SQL Content
# -------------------------------------------------------
GITHUB_RAW_URL = "https://raw.githubusercontent.com/navida-Informationssysteme/BC2Fabric-Toolbox/main/Accelerators/Finance-Reporting/finance-sql-views.sql"

print(f"Fetching SQL from: {GITHUB_RAW_URL}")
try:
    response = requests.get(GITHUB_RAW_URL)
    response.raise_for_status()
    sql_content = response.text
    print("SQL content fetched successfully.")
except Exception as e:
    print(f"Failed to fetch SQL file: {e}")
    raise

# -------------------------------------------------------
# 2. Connect to SQL Endpoint
# -------------------------------------------------------
# Get Entra ID (AAD) token
token = credentials.getToken("pbi")
token_bytes = token.encode("UTF-16LE")
token_struct = struct.pack(f'<I{len(token_bytes)}s', len(token_bytes), token_bytes)
SQL_COPT_SS_ACCESS_TOKEN = 1256 

# Construct Connection String using Global Parameters
conn_str = (
    f"Driver={{ODBC Driver 18 for SQL Server}};"
    f"Server={SQL_ENDPOINT},1433;"
    f"Database={DATABASE_NAME};"
    f"Encrypt=yes;"
    f"TrustServerCertificate=no;"
    f"Connection Timeout=30;"
)

print(f"Connecting to {DATABASE_NAME}...")

try:
    with pyodbc.connect(conn_str, attrs_before={SQL_COPT_SS_ACCESS_TOKEN: token_struct}, autocommit=True) as conn:
        cursor = conn.cursor()

        split_pattern = r'^\s*GO\s*;?\s*$'
        
        statements = re.split(split_pattern, sql_content, flags=re.IGNORECASE | re.MULTILINE)
        
        # Filter out empty strings
        statements = [stmt.strip() for stmt in statements if stmt.strip()]
        
        print(f"Found {len(statements)} primary batches to execute.")
        
        for i, stmt in enumerate(statements):
            sub_statements = [stmt]
            
            if "CREATE SCHEMA" in stmt.upper() and "CREATE OR ALTER VIEW" in stmt.upper():
                 # Lookahead split: Split right before "CREATE OR ALTER VIEW"
                 sub_statements = re.split(r'(?=CREATE\s+OR\s+ALTER\s+VIEW)', stmt, flags=re.IGNORECASE)

            for sub_stmt in sub_statements:
                clean_stmt = sub_stmt.strip()
                if not clean_stmt: continue
                
                print(f"Executing batch {i+1} part...")
                try:
                    cursor.execute(clean_stmt)
                except pyodbc.Error as e:
                    print(f"Error executing batch {i+1}:")
                    print(f"Query start: {clean_stmt[:200]}...") 
                    print(f"Error details: {e}")
                    raise 

        print("All views created successfully.")
        
except Exception as e:
    print("An error occurred during the database operation.")
    print(e)

# Finance Report Installation

This block performs the following actions:
1. Downloads the Power BI Report (.pbix) template from GitHub.
2. Imports it into the current Fabric Workspace.
3. Updates the **Servername** and **Company** parameters using the global configuration defined at the top of this notebook.
4. Triggers a dataset refresh.

In [ ]:
import requests
import time
import json

# We still import credentials for the token, but we don't use mssparkutils for the Workspace ID anymore
try:
    from notebookutils import credentials
except ImportError:
    from mssparkutils import credentials

# -------------------------------------------------------
# Configuration
# -------------------------------------------------------
# 1. GitHub URL (Must be the "Raw" version of the link)
GITHUB_PBIX_URL = "https://github.com/navida-Informationssysteme/BC2Fabric-Toolbox/blob/main/Accelerators/Finance-Reporting/BC2Fabric_Finance_App.pbix?raw=1"

# 2. The Name you want for the report in Fabric
REPORT_NAME = "BC2Fabric_Finance_App"

# 3. The Workspace ID (Using Spark Config - Most Reliable Method)
WORKSPACE_ID = spark.conf.get("trident.workspace.id")

# 4. Parameters to change after import { "ParameterName": "NewValue" }
# We use the variables defined in the Global Parameters cell at the top of the notebook
PARAMETERS_TO_UPDATE = {
    "Servername": SQL_ENDPOINT,
    "Company": COMPANY_NAME
}

# -------------------------------------------------------
# Helper Functions
# -------------------------------------------------------
def get_headers():
    """Get auth headers using the internal Fabric token"""
    token = credentials.getToken("pbi")
    return {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json"
    }

def poll_import_status(import_id, workspace_id):
    """Wait for the import to finish and return the new Dataset ID"""
    url = f"https://api.powerbi.com/v1.0/myorg/groups/{workspace_id}/imports/{import_id}"
    
    print("Waiting for import to complete...", end="")
    while True:
        response = requests.get(url, headers=get_headers())
        response.raise_for_status()
        data = response.json()
        state = data.get("importState")
        
        if state == "Succeeded":
            print(" Done!")
            return data["datasets"][0]["id"]
        elif state == "Failed":
            # Print full error for debugging
            print(f"\nImport Failed Details: {json.dumps(data, indent=2)}")
            raise Exception(f"Import Failed: {data.get('error')}")
        
        print(".", end="")
        time.sleep(2)

# -------------------------------------------------------
# Main Execution
# -------------------------------------------------------

# 1. Download PBIX
print(f"Downloading PBIX from: {GITHUB_PBIX_URL}")
file_response = requests.get(GITHUB_PBIX_URL)
file_response.raise_for_status()
print(f"Download complete. File size: {len(file_response.content) / 1024 / 1024:.2f} MB")

# 2. Upload (Import) to Workspace
import_url = (
    f"https://api.powerbi.com/v1.0/myorg/groups/{WORKSPACE_ID}/imports"
    f"?datasetDisplayName={REPORT_NAME}&nameConflict=CreateOrOverwrite"
)

files = {
    'file': (f'{REPORT_NAME}.pbix', file_response.content, 'application/octet-stream')
}
# Use a fresh token for the upload header
upload_headers = {"Authorization": f"Bearer {credentials.getToken('pbi')}"}

print(f"Uploading to Workspace ID: {WORKSPACE_ID}...")
upload_resp = requests.post(import_url, headers=upload_headers, files=files)
upload_resp.raise_for_status()
import_id = upload_resp.json()["id"]

# 3. Wait for Import & Get Dataset ID
dataset_id = poll_import_status(import_id, WORKSPACE_ID)
print(f"New Dataset ID: {dataset_id}")

# 4. Update Parameters
if PARAMETERS_TO_UPDATE:
    print("Updating Dataset Parameters...")
    
    # The UpdateParameters API requires the dataset to be 'ready'.
    # Sometimes it takes a split second after import succeeds to be writable.
    time.sleep(5) 
    
    update_url = f"https://api.powerbi.com/v1.0/myorg/groups/{WORKSPACE_ID}/datasets/{dataset_id}/Default.UpdateParameters"
    
    update_payload = {
        "updateDetails": [
            {"name": key, "newValue": str(value)} 
            for key, value in PARAMETERS_TO_UPDATE.items()
        ]
    }
    
    try:
        update_resp = requests.post(update_url, headers=get_headers(), json=update_payload)
        update_resp.raise_for_status()
        print("Parameters updated successfully.")
    except requests.exceptions.HTTPError as e:
        print(f"Failed to update parameters: {e.response.text}")
        # We don't raise here to allow the refresh to attempt anyway
    
    # 5. Trigger Refresh
    print("Triggering dataset refresh...")
    refresh_url = f"https://api.powerbi.com/v1.0/myorg/groups/{WORKSPACE_ID}/datasets/{dataset_id}/refreshes"
    refresh_resp = requests.post(refresh_url, headers=get_headers())
    refresh_resp.raise_for_status()
    print("Refresh started successfully.")

print("Process Complete.")